Langmuir-Hinshelwood model: Acceleration by Automated Rescaling of the Rate Constants.
--------------------------------------------------------------------------------------

The scale disparity issue is common in Kinetic Monte Carlo simulations. It emerges
as a result of the fact that some fundamental events or groups of them typically
occur at vastly different time scales; in other words, their rate constants can
span multiple orders of magnitude. In heterogeneous catalysis, there are typically
two groups: 1) very fast events that correspond to the species' surface diffusions
and 2) slow reactions that change their chemical identity. The latter group of events
is usually the one of interest because it allows the evaluation of the material's
catalytic activity. In contrast, the species' surface diffusion does not contribute
significantly to the net evolution of the slow reactions. But, as becomes the most
frequent step, it also becomes the limiting factor of the simulation progress,
considerably increasing the computational cost. This tutorial shows how to speed up
the calculation by several orders of magnitude without sacrificing precision by
automatically detecting and scaling the rate constants of fast reactions.

We will focus on the net reaction $\text{CO}+\frac{1}{2}\text{O}_2\longrightarrow \text{CO}_2$
that takes place at a catalyst's surface and whose reaction mechanism is described by
the Langmuir-Hinshelwood model. Because this model has four very fast processes
($CO$ and $O_2$ adsorption, and $O*$ and $CO*$ diffusion) and one slow process
($CO$ oxidation), it is an ideal prototype for demonstrating the benefits of the
automated rescaling of rate constants technique. Our ultimate goal is to investigate
how altering the relative percentage of the gas reactants $CO$ and $O_2$ (at a specific
temperature and pressure) affect the rate of $CO_2$ production under steady state
conditions. This example is inspired on Zacros tutorial
[What's KMC All About and Why Bother](https://zacros.org/tutorials/12-about-kinetic-monte-carlo?showall=1).

Let's start! The first step is to import all packages we need:

In [1]:
import multiprocessing
import numpy
import scm.plams
import scm.pyzacros as pz
import scm.pyzacros.models

Then, we initialize the **pyZacros** environment. 

In [2]:
scm.pyzacros.init()

PLAMS working folder: /home/aguirre/Develop/pyzacros/examples/ZiffGulariBarshad/plams_workdir.002


Notice this command created the directory where all **Zacros** input and output files
will be stored if they are needed for future reference (``plams_workdir`` by default).
Typically, the user doesn't need to use these files.

This calculation necessitates a significant computational effort. On a typical laptop,
it should take less than an hour to complete. So, in order to speed things up, we'll
use the ``plams.JobRunner`` class to run as many parallel instances as possible. In this
case, we choose to use the maximum number of simultaneous processes (``maxjobs``) equal
to the number of processors in the machine. 

In [3]:
maxjobs = multiprocessing.cpu_count()
scm.plams.config.default_jobrunner = scm.plams.JobRunner(parallel=True, maxjobs=maxjobs)
print('Running up to {} jobs in parallel simultaneously'.format(maxjobs))

Running up to 8 jobs in parallel simultaneously


Now, we initialize our Langmuir-Hinshelwood model, which by luck is available as a predefined model in pyZacros,

In [4]:
lh = pz.models.LangmuirHinshelwood()

Then, we set up the Zacros calculation. All parameters are set using a ``Setting``
object. To begin, we define the physical parameters: ``temperature`` (in K), and
``pressure`` (in bar). The calculation parameters are then set: ``species numbers``
(in s) determines how frequently information about the number of gas and surface
species will be stored, ``max time`` (in s) specifies the maximum allowed simulated
time, and "random seed" specifies the random seed to make the calculation precisely
reproducible. Keep in mind that ``max time`` defines the calculation's stopping
criterion, and it is the parameter that will be controlled later to achieve the
steady-state configuration. Finally, we create the ``ZacrosJob``, which uses the
parameters we just defined as well as the Langmuir-Hinshelwood model's lattice,
mechanism, and cluster expansion. Notice we do not run this job, we use it as a
reference for the steady-state calculation described below.

In [5]:
z_sett = pz.Settings()
z_sett.temperature = 500.0
z_sett.pressure = 1.000
z_sett.species_numbers = ('time', 1.0e-5)
z_sett.max_time = 100*1.0e-5
z_sett.random_seed = 1609

z_job = pz.ZacrosJob( settings=z_sett, lattice=lh.lattice,
                      mechanism=lh.mechanism,
                      cluster_expansion=lh.cluster_expansion )

Hmmmmm

In [6]:
ss_sett = pz.Settings()
#ss_sett.turnover_frequency.confidence = 0.98
#ss_sett.turnover_frequency.confidence = 0.99 --> Funciona???? para dejar el default?
ss_sett.turnover_frequency.confidence = 0.5
ss_sett.turnover_frequency.ignore_nbatch = 5
# ss_sett.turnover_frequency.ignore_nbatch = 1 --> Funciona??> para dejar el default?
ss_sett.nreplicas = 4
ss_sett.scaling.enabled = 'T'

ss_params = pz.ZacrosSteadyStateJob.Parameters()
ss_params.add( 'max_time', 'restart.max_time',
                2*z_sett.max_time*( numpy.arange(50)+1 )**2 )

ss_job = pz.ZacrosSteadyStateJob( settings=ss_sett, reference=z_job,
                                  parameters=ss_params )

[18.01|17:53:30] JOB plamsjob Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4


In [7]:
ps_params = pz.ZacrosParametersScanJob.Parameters()
#ps_params.add( 'x_CO', 'molar_fraction.CO', numpy.linspace(0.0, 1.0, 21) )
ps_params.add( 'x_CO', 'molar_fraction.CO', numpy.linspace(0.0, 1.0, 3) )
ps_params.add( 'x_O2', 'molar_fraction.O2', lambda params: 1.0-params['x_CO'] )

ps_job = pz.ZacrosParametersScanJob( reference=ss_job, parameters=ps_params )

[18.01|17:53:30] JOB plamsjob_ps_cond000 Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4
[18.01|17:53:30] JOB plamsjob_ps_cond001 Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4
[18.01|17:53:30] JOB plamsjob_ps_cond002 Steady State Convergence: Using nbatch=20,confidence=0.5,ignore_nbatch=5,nreplicas=4


In [8]:
results = ps_job.run()

if not ps_job.ok():
    print('Something went wrong!')

[18.01|17:53:30] JOB plamsjob STARTED
[18.01|17:53:30] Waiting for job plamsjob to finish
[18.01|17:53:30] JOB plamsjob RUNNING
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond000 STARTED
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond001 STARTED
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond002 STARTED
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond000 RUNNING
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond001 RUNNING
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond000 Scaling: Using partial_equilibrium_index_threshold=0.1,upper_bound=100
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond002 RUNNING
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond001 Scaling: Using partial_equilibrium_index_threshold=0.1,upper_bound=100
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond000/plamsjob_ps_cond000_ss_scaling STARTED
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond002 Scaling: Using partial_equilibrium_index_threshold=0.1,upper_bound=100
[18.01|17:53:30] JOB plamsjob/plamsjob_ps_cond001/plamsjob_ps_cond001_ss

In [9]:
x_CO = []
ac_O = []
ac_CO = []
TOF_CO2 = []

results_dict = results.turnover_frequency()
results_dict = results.average_coverage( last=10, update=results_dict )

for i in range(len(results_dict)):
    x_CO.append( results_dict[i]['x_CO'] )
    ac_O.append( results_dict[i]['average_coverage']['O*'] )
    ac_CO.append( results_dict[i]['average_coverage']['CO*'] )
    TOF_CO2.append( results_dict[i]['turnover_frequency']['CO2'] )

print( '------------------------------------------------' )
print( '%4s'%'cond', '%8s'%'x_CO', '%10s'%'ac_O', '%10s'%'ac_CO', '%12s'%'TOF_CO2' )
print( '------------------------------------------------' )
for i in range(len(x_CO)):
    print( '%4d'%i, '%8.2f'%x_CO[i], '%10.6f'%ac_O[i], '%10.6f'%ac_CO[i], '%12.6f'%TOF_CO2[i] )

------------------------------------------------
cond     x_CO       ac_O      ac_CO      TOF_CO2
------------------------------------------------
   0     0.00   0.539562   0.000000     0.000000
   1     0.50   0.447031   0.271031   307.500000
   2     1.00   0.000000   0.151219     0.000000


In [10]:
scm.pyzacros.finish()

[18.01|17:59:12] PLAMS run finished. Goodbye


$$
\theta_\text{CO} = \frac{ K_\text{CO}P_\text{CO} }{ 1 + K_\text{CO}P_\text{CO} + \sqrt{K_{\text{O}_2}P_\text{O}} } \\
\theta_\text{O} = \frac{ \sqrt{K_{\text{O}_2}P_\text{O}} }{ 1 + K_\text{CO}P_\text{CO} + \sqrt{K_{\text{O}_2}P_\text{O}} } \\[7mm]
\text{TOF}_{\text{CO}_2} = 6 \, k_\text{oxi}\theta_\text{CO}\theta_\text{O}
$$